# Read and clean a single file

In [1]:
import os
import sys
import logging
import pandas as pd

from pandas_profiling import ProfileReport


project_dir = os.path.dirname(os.path.abspath('.')) # get the path of the project, if doesn't work use os.getcwd() instead and use rfind to find the project dir
sys.path.append(os.path.join(project_dir, 'src', 'data'))

from data_manipulator import get_input_data

bucket = 'ctovar_espbigdata'
directory = 'BigData'

## Read Metadata
-----
General info for each relevant column

In [2]:
file = 'gs://{}/{}/data/metadatos-llamadas-urg-y-emer.csv'.format(bucket, directory)
df_metadata = pd.read_csv(file, encoding='latin1', sep=';')
df_metadata

,NOMBRE,DESCRIPCION
0,FECHA_INCIDENTE,Es la fecha el cual se registra la llamada del...
1,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,Es la fecha el cual se inicia el desplazamient...
2,CODIGO LOCALIDAD,Es el código de las 20 localidades de la ciuda...
3,LOCALIDAD,Es la localidad donde sucede el incidente.
4,EDAD,La edad del paciente.
5,UNIDAD,"La descripción de la edad si es en horas, días..."
6,GENERO,Es la distinción de genero del paciente
7,RED,Es la localización a nivel bogota de la red de...
8,TIPO_INCIDENTE,Es la descripción inicial que tipifica el cent...
9,PRIORIDAD,Es la tipificación según la prioridad del inci...


The final data must contains this and only this columns with that specific headers in order to standarize the data, this is a type of data dictionary but it also must include the data types (**schema**) of each field (**column**)

## Get raw data

In [3]:
get_input_data?

Signature:
get_input_data(
    bucket='esp-big-data',
    initial_directory='BigData',
    filename='datos-abiertos-agosto-2019.csv',
)
Docstring:
Read a csv file in a bucket of GCS, the file must use latin1 encoding and the separator is a semicolon (;)

Args:
    bucket (str, optional): Name of the bucket. Defaults to 'esp-big-data'.
    initial_directory (str, optional): project directory. Defaults to 'BigData'.
    filename (str, optional):csv file to read. Defaults to 'datos-abiertos-agosto-2019.csv'.

Returns:
    pandas.dataframe: dataframe with the raw data
File:      ~/ESEIT_BigData/src/data/data_manipulator.py
Type:      function


In [4]:
raw_data = get_input_data(
    bucket = bucket,
    initial_directory = directory,
    filename = 'datos-abiertos-octubre-2019.csv'
)
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18195 entries, 0 to 18194
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   NUMERO_INCIDENTE     18195 non-null  int64 
 1   FECHA_DESPACHO_518   18195 non-null  object
 2   CODIGO DE LOCALIDAD  18195 non-null  int64 
 3   LOCALIDAD            18195 non-null  object
 4   EDAD                 18195 non-null  object
 5   UNIDAD               9503 non-null   object
 6   GENERO               9543 non-null   object
 7   RED                  18195 non-null  object
 8   TIPO_INCIDENTE       18195 non-null  object
 9   PRIORIDAD            18195 non-null  object
dtypes: int64(2), object(8)
memory usage: 1.4+ MB
None


It seems that this file contains already the fields, but need to rename them

### Rename Columns

In [5]:
# Don't let white spaces between headers to avoid errors during the pipeline
# This is sometimes called a recipe for etl
raw_data.rename(
    columns = {
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'TIPO DE INCIDIDENT'                : 'TIPO_INCIDENTE',
    },
    inplace=True
)
raw_data.head()

,NUMERO_INCIDENTE,FECHA_DESPACHO_518,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,868419196,2019-10-01 00:06:29,9,Fontibon,SIN_DATO,NaN,NaN,Norte,Dolor torácico,ALTA
1,868421196,2019-10-01 00:19:40,10,Engativa,67,Años,MASCULINO,Norte,Heridos,ALTA
2,868438196,2019-10-01 00:13:20,4,San Cristobal,SIN_DATO,NaN,NaN,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,944374194,2019-10-01 05:30:23,16,Puente Aranda,SIN_DATO,NaN,NaN,Privada,Convulsiones,ALTA
4,868476196,2019-10-01 00:42:45,5,Usme,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA


### Check values per field
--------
1. LOCALIDAD
2. GENERO

In [6]:
raw_data['LOCALIDAD'].value_counts(dropna=False)

 Kennedy               2546
 Engativa              1791
 Suba                  1742
 Bosa                  1334
 Ciudad Bolivar        1141
 Puente Aranda          989
 Rafael Uribe Uribe     949
Usaquen                 888
 Fontibon               863
San Cristobal           844
 Chapinero              763
 Los Martires           697
 Santa Fe               689
 Barrios Unidos         680
 Usme                   609
 Tunjuelito             589
 Teusaquillo            536
 Antonio Nariño         419
 La Candelaria          120
 Sumapaz                  6
Name: LOCALIDAD, dtype: int64

It seems that in some fields exist a white space at the begining of the values, probably at the end too, lets clean them all

In [11]:
# Let's use the function strip(), this property works for string objects in python that removes whitespaces
raw_data['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               2546
Engativa              1791
Suba                  1742
Bosa                  1334
Ciudad Bolivar        1141
Puente Aranda          989
Rafael Uribe Uribe     949
Usaquen                888
Fontibon               863
San Cristobal          844
Chapinero              763
Los Martires           697
Santa Fe               689
Barrios Unidos         680
Usme                   609
Tunjuelito             589
Teusaquillo            536
Antonio Nariño         419
La Candelaria          120
Sumapaz                  6
Name: LOCALIDAD, dtype: int64

In [6]:
raw_data['LOCALIDAD'] = raw_data['LOCALIDAD'].apply(lambda x: x.strip()) # we assign the output of the function apply to the same column

In [7]:
raw_data.head()

,NUMERO_INCIDENTE,FECHA_DESPACHO_518,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,868419196,2019-10-01 00:06:29,9,Fontibon,SIN_DATO,NaN,NaN,Norte,Dolor torácico,ALTA
1,868421196,2019-10-01 00:19:40,10,Engativa,67,Años,MASCULINO,Norte,Heridos,ALTA
2,868438196,2019-10-01 00:13:20,4,San Cristobal,SIN_DATO,NaN,NaN,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,944374194,2019-10-01 05:30:23,16,Puente Aranda,SIN_DATO,NaN,NaN,Privada,Convulsiones,ALTA
4,868476196,2019-10-01 00:42:45,5,Usme,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA


In [9]:
#col = 'GENERO'
#raw_data[col] = raw_data[col].apply(lambda x: x.strip())
#raw_data[col].value_counts(dropna=False)   # here we could apply the same technique as before to avoid errors

In [10]:
col = 'TIPO_INCIDENTE'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Accidente de tránsito con heridos/Muertos    6778
Inconsciente/Paro Cardiorrespiratorio        1768
Dificultad Respiratoria                      1497
Enfermo                                      1276
Convulsiones                                 1130
Heridos                                       828
Dolor torácico                                735
Intento de suicidio                           668
Lesiones personales                           657
Trastorno mental                              654
Accidente cerebro vascular                    396
Patología Gineco - obstétrica                 378
Síntomas gastrointestinales                   375
Caída                                         322
Intoxicaciones                                179
Ideas de suicidio                             111
SIN_DATO                                       60
Incendio estructural                           51
Muerte Natural                                 39
Violencia Sexual                               32


Here this columns contains a null value, aka NaN (this is numeric value that is Null), lets replace them first
* NaN --> SIN_DATO

In [11]:
raw_data[col].fillna('SIN_DATO').value_counts()

Accidente de tránsito con heridos/Muertos    6778
Inconsciente/Paro Cardiorrespiratorio        1768
Dificultad Respiratoria                      1497
Enfermo                                      1276
Convulsiones                                 1130
Heridos                                       828
Dolor torácico                                735
Intento de suicidio                           668
Lesiones personales                           657
Trastorno mental                              654
Accidente cerebro vascular                    396
Patología Gineco - obstétrica                 378
Síntomas gastrointestinales                   375
Caída                                         322
Intoxicaciones                                179
Ideas de suicidio                             111
SIN_DATO                                       60
Incendio estructural                           51
Muerte Natural                                 39
Violencia Sexual                               32


# Convert to datetime data type columns must be
________________________________________________

In [12]:
raw_data["FECHA_DESPACHO_518"]= pd.to_datetime(raw_data["FECHA_DESPACHO_518"])
raw_data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18195 entries, 0 to 18194
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   NUMERO_INCIDENTE    18195 non-null  int64         
 1   FECHA_DESPACHO_518  18195 non-null  datetime64[ns]
 2   CODIGO_LOCALIDAD    18195 non-null  int64         
 3   LOCALIDAD           18195 non-null  object        
 4   EDAD                18195 non-null  object        
 5   UNIDAD              9503 non-null   object        
 6   GENERO              9543 non-null   object        
 7   RED                 18195 non-null  object        
 8   TIPO_INCIDENTE      18195 non-null  object        
 9   PRIORIDAD           18195 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 1.4+ MB


## Remove duplicates

In [13]:
print('Number of rows before cleaning:', raw_data.shape[0] )
raw_data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data.shape[0] )

Number of rows before cleaning: 18195
Number of rows after cleaning: 16559


# Profile of the Data
------
first install pandas profiling, from a terminal

````
pip install pandas-profiling
````

In [14]:
profile = ProfileReport(df=raw_data)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_octubre_2019.html')

Summarize dataset:   0%|          | 0/24 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Form the initial report of the file we see that are duplicates rows in the table, we need to subtract them

### Save the final table
use to_csv to save the table

In [15]:
# saving in a local directory
raw_data.to_csv(project_dir + '/data/processed/' +'clean_llamadas_123_octubre_2019.csv', encoding='latin1', sep=';', index=False)